# Mesh creation in serial and parallel
Author: Jørgen S. Dokken

In this tutorial we will consider the first important class in DOLFINx, the `dolfinx.mesh.Mesh` class.

A mesh consists of a set of cells. These cells can be intervals, triangles, quadrilaterals, hexahedra or tetrahedra.
Each cell is described by a set of coordinates, and its connectivity.

## Mesh creation from Numpy arrays
For instance, let us consider a unit square. If we want to discretize it with triangular elements, we could create the set of vertices as a $(4\times 2)$ numpy array

In [ ]:
import numpy as np
tri_points = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])

Next, we have to decide on how we want to create the two triangles in the mesh. Let's choose the first cell to consist of vertices $0,1,3$ and the second cell consist of vertices $0,2,3$

In [ ]:
triangles = np.array([[0, 1, 3], [0, 2, 3]])

We note that for triangular cells, we could order the vertices in any order say `[[1,3,0],[2,0,3]]`, and the mesh would equivalent.
Some finite element software reorder cells to ensure consistent integrals over interior facets.
In DOLFINx, another strategy has been chosen, see {cite}`10.1145/3524456` for more details.

Let's consider the unit square again, but this time we want to discretize it with two quadrilateral cells.

In [ ]:
quad_points = np.array([[0.0, 0.0],[0.3, 0.0], [1.0, 0.0], [0.0, 1.0], [0.4, 1.0], [1.0, 1.0]])
quadrilaterals = np.array([[0, 1, 3, 4], [1, 2, 4, 5]])

Note that we do not parse the quadrilateral cell in a clockwise or counter-clockwise fashion.
Instead, we are using a tensor product ordering.
The ordering of the sub entities all cell types used in DOLFINx can be found at [Basix supported elements](https://github.com/FEniCS/basix/#supported-elements).
We also note that this unit square mesh has non-affine elements.

Next, we would like to generate the mesh used in DOLFINx.
To do so, we need to generate the coordinate element.
This is the paramterisation of each and every element, and the only way of going between the physical element and the reference element.
We will denote any coordinate on the reference element as $\mathbf{X}$,
and any coordinate in the physical element as $\mathbf{x}$,
with the coodinate map for a given cell $M$ such that $\mathbf{x} = M(\mathbf{X})$.

We can write 

$$
\begin{align}
M(\mathbf{X})= \sum_{i=0}^{\text{num vertices}} \mathbf{v}_i\phi_i(\mathbf{X}
\end{align})
$$

where $\mathbf{v}_i$ is the $i$th vertex of a cell and $\phi_i$ are the basis functions as specifed at [DefElement P1 triangles](https://defelement.com/elements/examples/triangle-Lagrange-1.html) and
[DefElement Q1 quadrilaterals](https://defelement.com/elements/examples/quadrilateral-Q-1.html).

In DOLFINx, we use [UFL (the Unified Form Language)](https://github.com/FEniCS/ufl/) to define finite elements.
Therefore we create the `ufl.Mesh`.

In [ ]:
import ufl
ufl_tri = ufl.Mesh(ufl.VectorElement("Lagrange", ufl.triangle, 1))
ufl_quad = ufl.Mesh(ufl.VectorElement("Lagrange", ufl.quadrilateral, 1))

This is all the input we need to a DOLFINx mesh

In [ ]:
import dolfinx
from mpi4py import MPI

quad_mesh = dolfinx.mesh.create_mesh(
    MPI.COMM_WORLD, quadrilaterals, quad_points, ufl_quad)
tri_mesh = dolfinx.mesh.create_mesh(
    MPI.COMM_WORLD, triangles, tri_points, ufl_tri)

The only input to this function we have not covered so far is `MPI.COMM_WORLD`, which is an MPI communicator.

### MPI Communication
When we run a python code with `python3 name_of_file.py`, we execute python on a single process on the computer. However, if we launch the code with `mpirun -n N python3 name_of_file.py`, we execute the code on `N` processes at the same time. The `MPI.COMM_WORLD` is the communicator among the `N` processes, which can be used to send and receive data. If we use `MPI.COMM_SELF`, the communicator will not communicate with other processes.
When we run in serial, `MPI.COMM_WORLD` is equivalent to `MPI.COMM_SELF`.

Two important values in the MPI-communicator is its `rank` and `size`.
If we run this in serial on either of the communicators above, we get


In [ ]:
print(f"{MPI.COMM_WORLD.rank=} {MPI.COMM_WORLD.size=}")
print(f"{MPI.COMM_SELF.rank=} {MPI.COMM_SELF.size=}")

In jupyter noteboooks, we use [ipyparallel](https://ipyparallel.readthedocs.io/en/latest/) to start a cluster and connect to two processes, which we can execute commands on using the magic `%%px` at the top of each cell. See [%%px Cell magic](https://ipyparallel.readthedocs.io/en/latest/tutorial/magics.html#px-cell-magic) for more details.

```{note}
When starting a cluster, we do not carry ower any modules or variables from the previously executed code in the script.
```